In [146]:
#need to read in a layer
#test a probe and write the results 

#multiple layers? --> last three of hte xl embeddings 

In [147]:
import pandas as pd
df = pd.read_csv('./data/animals_true_false.csv')
source_labels = df['label']

In [148]:
# import os

# # Specify the directory path
# directory_path = "data/embeddings/llama3/basic"

# # Get all files in the directory
# all_files = os.listdir(directory_path)

# for filename in all_files:
#     # Check if the filename contains square brackets
#     if "[" in filename or "]" in filename:
#         # Construct the old and new file paths
#         old_path = os.path.join(directory_path, filename)
#         new_filename = filename.replace("[", "").replace("]", "")
#         new_path = os.path.join(directory_path, new_filename)

#         # Rename the file
#         os.rename(old_path, new_path)
#         print(f"Renamed {filename} to {new_filename}")

In [149]:
import os
import numpy as np

# Set the folder path
folder_path = "data/embeddings/llama3/basic"

merged_embeddings = []
basic_embeddings = []
llama_embeddings = []
layer1 = []
layer2 = []
layer3 = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is a numpy file
    if filename.endswith(".npy"):
        # Load the embedding data from the file
        embedding_data = np.load(file_path)
        
        # Check if the filename contains "merged"
        if "-1" in filename:
            layer1.append(embedding_data)
        elif "-2" in filename:
            layer2.append(embedding_data)
        else:
            layer3.append(embedding_data)
        llama_embeddings.append(embedding_data)




In [150]:
import numpy as np

# Concatenate layers along the embedding dimension
combined_embeddings = np.concatenate([layer2, layer3], axis=1)
print(combined_embeddings.shape)  # Output: (1008, 12288)

(1008, 8192)


In [161]:
def add_noise(embeddings, noise_level=0.1):
    noise = np.random.normal(0, noise_level, combined_embeddings.shape)
    return embeddings + noise

augmented_embeddings = add_noise(combined_embeddings)

In [162]:
# from itertools import combinations

# def generate_combinations(x, y):
#     """
#     Generates all combinations of numbers from x to y.
    
#     Args:
#         x (int): The lower bound of the range (inclusive).
#         y (int): The upper bound of the range (inclusive).
        
#     Returns:
#         list: A list of tuples, where each tuple represents a combination
#         of numbers from x to y.
#     """
#     nums = list(range(x, y+1))  # Generate a list of numbers from x to y
#     combinations_list = []
    
#     # Generate combinations of different lengths
#     for length in range(1, len(nums)+1):
#         combs = combinations(nums, length)
#         combinations_list.extend(combs)
    
#     return combinations_list

In [174]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def probe(embeddings, labels, layer_list, input_dim=1600):
  X = np.array(embeddings)  
  y = np.array(labels)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Define and compile model
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, y_test)

  layer_str = '-'.join(map(str, layer_list))
  save_data(test_loss, test_acc, layer_str)

def save_data(test_loss, test_acc, layer):
  write_data = f'{layer}: {test_loss} {test_acc}'
  with open('./statistics/Layer_Data/dump2', 'a') as file:
    file.write(f'{write_data}\n')

In [164]:
# indices = generate_combinations(0,48)

In [165]:
# indices

In [166]:
def parse_embeddings(embeddings_list, index_list):
    flat_list = []
    for index in index_list:
        flat_list.extend(embeddings_list[index])
        print(len(flat_list))
    return flat_list  

In [167]:
indices = [(2,6)]

In [168]:
for i in range(32):
  for index_list in indices:
    #construct embeddings for this list 
    #create the label amount by multiplying 
    #embeddings = parse_embeddings(llama_embeddings, index_list)
    embeddings = combined_embeddings
    labels = df['label'] 
    probe(embeddings, labels=labels, layer_list=index_list)
    #augmented_accuracy = 
    #probe(augmented_embeddings, labels, layer_list = index_list)

Epoch 1/10
21/21 [==============================] - 1s 8ms/step - loss: 1.0217 - accuracy: 0.4984 - val_loss: 0.6987 - val_accuracy: 0.5370
Epoch 2/10
21/21 [==============================] - 0s 3ms/step - loss: 0.7545 - accuracy: 0.4953 - val_loss: 0.6927 - val_accuracy: 0.5185
Epoch 3/10
21/21 [==============================] - 0s 3ms/step - loss: 0.7028 - accuracy: 0.4845 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 4/10
21/21 [==============================] - 0s 4ms/step - loss: 0.6952 - accuracy: 0.5047 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 5/10
21/21 [==============================] - 0s 4ms/step - loss: 0.6935 - accuracy: 0.5047 - val_loss: 0.6932 - val_accuracy: 0.5062
Epoch 6/10
21/21 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.5047 - val_loss: 0.6925 - val_accuracy: 0.5309
Epoch 7/10
21/21 [==============================] - 0s 4ms/step - loss: 0.6981 - accuracy: 0.5124 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 8/10
21/21 [==

In [180]:
layer_combinations = [
    [1],  # Layer 1 only
    [2],  # Layer 2 only
    [3],  # Layer 3 only
    [1, 2],  # Layers 1 and 2
    [1, 3],  # Layers 1 and 3
    [2, 3],  # Layers 2 and 3
    [1, 2, 3]  # All layers
]

for i in range(32):
    for index_list in layer_combinations:
    # Concatenate the selected layers
        selected_embeddings = []
        for index in index_list:
            if index == 1:
                selected_embeddings.append(layer1)
            elif index == 2:
                selected_embeddings.append(layer2)
            elif index == 3:
                selected_embeddings.append(layer3)
        combined_embeddings = np.concatenate(selected_embeddings, axis=1)
        print(len(selected_embeddings))

        # Probe with the combined embeddings
        layer_str = f"Layers {', '.join(map(str, index_list))}"
        #probe(combined_embeddings, labels, index_list)

1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
1
1
1
2
2
2
3
